# Train a correct segment network

In [1]:
import torch
torch.manual_seed(237)

/home/ke/anaconda3/envs/livecell/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pathlib import Path
import pandas as pd
train_dir = Path("./notebook_results/a549_ccp_vim/train_data/")
train_csv = train_dir / "train_data.csv"
train_df = pd.read_csv(train_csv)
train_df[:2]

,raw,seg,gt,raw_seg,scale
0,notebook_results/a549_ccp_vim/train_data/raw/i...,notebook_results/a549_ccp_vim/train_data/augme...,notebook_results/a549_ccp_vim/train_data/gt/im...,notebook_results/a549_ccp_vim/train_data/seg/i...,-0.300000
1,notebook_results/a549_ccp_vim/train_data/raw/i...,notebook_results/a549_ccp_vim/train_data/augme...,notebook_results/a549_ccp_vim/train_data/gt/im...,notebook_results/a549_ccp_vim/train_data/seg/i...,-0.233333


In [3]:
train_df.shape

(560, 5)

In [4]:
raw_img_paths = list(train_df["raw"])
seg_mask_paths = list(train_df["seg"])
gt_mask_paths = list(train_df["gt"])
raw_seg_paths = list(train_df["raw_seg"])
scales = list(train_df["scale"])

In [5]:
from livecell_tracker.model_zoo.segmentation.sc_correction_dataset import CorrectSegNetDataset
dataset = CorrectSegNetDataset(raw_img_paths, seg_mask_paths, gt_mask_paths, raw_seg_paths=raw_seg_paths, scales=scales)

/home/ke/anaconda3/envs/livecell/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/ke/anaconda3/envs/livecell/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


In [6]:
train_input_tuples = list(zip(raw_img_paths, seg_mask_paths, gt_mask_paths, raw_seg_paths, scales))
train_input_tuples[:1]

[('notebook_results/a549_ccp_vim/train_data/raw/img-1_seg-14.tif',
  'notebook_results/a549_ccp_vim/train_data/augmented_seg/img-1_seg-14_aug-0.tif',
  'notebook_results/a549_ccp_vim/train_data/gt/img-1_seg-14.tif',
  'notebook_results/a549_ccp_vim/train_data/seg/img-1_seg-14.tif',
  -0.3)]

In [7]:
for data in list(zip(*train_input_tuples)):
    print(data[0])

notebook_results/a549_ccp_vim/train_data/raw/img-1_seg-14.tif
notebook_results/a549_ccp_vim/train_data/augmented_seg/img-1_seg-14_aug-0.tif
notebook_results/a549_ccp_vim/train_data/gt/img-1_seg-14.tif
notebook_results/a549_ccp_vim/train_data/seg/img-1_seg-14.tif
-0.3


pytorch transform example

In [8]:
from torchvision import transforms
train_transforms = transforms.Compose(
    [
        # transforms.Resize((412, 412)),
        transforms.RandomCrop((412, 412)),
        transforms.RandomHorizontalFlip(412, 412, padd_if_needed=True),
        transforms.RandomAffine(degrees=360, translate=(1, 1), scale=(0.5, 1.5)),
    ]
)
train_transforms(dataset[0]["input"]).shape, train_transforms(dataset[0]["input"])[0, 0, 0], dataset[0]["input"].shape,dataset[0]["input"][0, 0, 0]

(torch.Size([3, 412, 412]),
 tensor(15162.4404),
 torch.Size([3, 441, 427]),
 tensor(15174.))

In [9]:
import torch
import torch.utils.data
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import TQDMProgressBar

from livecell_tracker.model_zoo.segmentation.sc_correction import CorrectSegNet

# TODO: decide augment in dataset object or in model?
img_paths, mask_paths, gt_paths, seg_paths, scales= list(zip(*train_input_tuples))
dataset = CorrectSegNetDataset(img_paths, mask_paths, gt_paths, raw_seg_paths=seg_paths, scales=scales, transform=train_transforms)

train_sample_num = int(len(dataset) * 0.8)
val_sample_num = len(dataset) - train_sample_num
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_sample_num, val_sample_num])

model = CorrectSegNet(train_input_paths=train_input_tuples, num_workers=1, batch_size=2, train_transforms=train_transforms, train_dataset=train_dataset, val_dataset=val_dataset, test_dataset=val_dataset)
trainer = Trainer(gpus=1, max_epochs=500)
trainer.fit(model)

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


/home/ke/anaconda3/envs/livecell/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/ke/anaconda3/envs/livecell/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/ke/anac

Sanity Checking: 0it [00:00, ?it/s]

/home/ke/anaconda3/envs/livecell/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/ke/anaconda3/envs/livecell/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 64:   7%|▋         | 20/280 [00:09<02:01,  2.15it/s, loss=0.0021, v_num=33, val_loss=0.00201, val_acc=0.999]  